In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [ ]:
# Determine the number of unique values in each column.
application_df.nunique(axis=0)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_values = application_df['APPLICATION_TYPE'].value_counts()

### OPTIMIZATION #1: Reduce Application Types further (<1000)  ###
### OPTIMIZATION #5: Increase Application Types further (<100) ###
application_types_to_replace = app_values[app_values < 1000].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: count, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classifications = application_df['CLASSIFICATION'].value_counts()

### OPTIMIZATION #6: Increase Classification Types further (<100) ###
classifications_to_replace = classifications[classifications < 200].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: count, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_df = pd.get_dummies(application_df)
dummy_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,True,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,True,False,True,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,True,False,False,False,True,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [ ]:
# Split our preprocessed data into our features and target arrays
y = dummy_df['IS_SUCCESSFUL']
X = dummy_df.drop(['IS_SUCCESSFUL'], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu"))

### OPTIMIZATION #4: Increase units in second layer and remove 3rd layer ###
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu"))

### OPTIMIZATION #2: Add a 3rd hidden layer  ###
#nn.add(tf.keras.layers.Dense(units=40, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

build_x = tf.ones((1,42))
build_y = nn(build_x)

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (1, 80)                   3440      
                                                                 
 dense_28 (Dense)            (1, 80)                   6480      
                                                                 
 dense_29 (Dense)            (1, 1)                    81        
                                                                 
Total params: 10001 (39.07 KB)
Trainable params: 10001 (39.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='weights-optimized.h5',
                                     save_weights_only=True, period=5)

### OPTIMIZATION #3: Moved epochs to 120  ###
fit_model = nn.fit(X_train_scaled, y_train, epochs=120, callbacks=[checkpoint])

Epoch 1/120
858/858 [==============================] - 3s 2ms/step - loss: 0.5741 - accuracy: 0.7181
Epoch 2/120
858/858 [==============================] - 2s 2ms/step - loss: 0.5601 - accuracy: 0.7249
Epoch 3/120
858/858 [==============================] - 2s 2ms/step - loss: 0.5565 - accuracy: 0.7286
Epoch 4/120
858/858 [==============================] - 2s 2ms/step - loss: 0.5544 - accuracy: 0.7283
Epoch 5/120
858/858 [==============================] - 2s 2ms/step - loss: 0.5538 - accuracy: 0.7298
Epoch 6/120
858/858 [==============================] - 2s 2ms/step - loss: 0.5523 - accuracy: 0.7309
Epoch 7/120
858/858 [==============================] - 3s 3ms/step - loss: 0.5510 - accuracy: 0.7315
Epoch 8/120
858/858 [==============================] - 2s 2ms/step - loss: 0.5509 - accuracy: 0.7308
Epoch 9/120
858/858 [==============================] - 2s 2ms/step - loss: 0.5501 - accuracy: 0.7309
Epoch 10/120
858/858 [==============================] - 2s 2ms/step - loss: 0.5493 - accura

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - loss: 0.5671 - accuracy: 0.7375 - 585ms/epoch - 3ms/step
Loss: 0.5671241283416748, Accuracy: 0.7374635338783264


In [ ]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
